# Testing how DataforCEO Keywords API works

23.01.2024 - Enzo Yamamura

## Credentials and libs

In [1]:
from dotenv import load_dotenv
from client import RestClient
import pandas as pd
import plotly.express as px
import datetime
import os

load_dotenv()

# Data for SEO credentials
login = os.getenv("login")
password = os.getenv("password")

## Post Request

**Use Case:**

 "...oito SUVs compactos com preços entre R$ 140.000 e R$ 170.000 – a oferta do segmento contempla uma faixa bem mais ampla, considerando versões e conteúdos..."

 "Participam: Chevrolet Tracker Premier 1.2, Fiat Fastback Limited Edition, Honda HR-V EXL, Hyundai Creta N-Line, Jeep Renegade Longitude T270, Nissan Kicks Ex-clusive Pack, Renault Duster Iconic 1.3 TC..."

Leia mais em: https://quatrorodas.abril.com.br/testes/comparativo-tracker-fastback-hr-v-creta-renegade-kicks-duster-t-cross

In [2]:
# Keywords
keywords = ["Chevrolet Tracker Premier 1.2", "Fiat Fastback Limited Edition", "Honda HR-V EXL", "Hyundai Creta N-Line",
            "Jeep Renegade Longitude T270", "Nissan Kicks Ex-clusive Pack", "Renault Duster Iconic 1.3 TC"]
keywords = [k.lower() for k in keywords]
keywords

['chevrolet tracker premier 1.2',
 'fiat fastback limited edition',
 'honda hr-v exl',
 'hyundai creta n-line',
 'jeep renegade longitude t270',
 'nissan kicks ex-clusive pack',
 'renault duster iconic 1.3 tc']

In [3]:
client = RestClient(login, password)
post_data = dict()
# simple way to set a task
post_data[len(post_data)] = dict(
    location_name="Brazil",
    keywords=keywords
)


response = client.post(
    "/v3/keywords_data/google_ads/search_volume/task_post", post_data)
# you can find the full list of the response codes here https://docs.dataforseo.com/v3/appendix/errors
if response["status_code"] == 20000:
    print(response)
    # do something with result
else:
    print("error. Code: %d Message: %s" %
          (response["status_code"], response["status_message"]))

{'version': '0.1.20231117', 'status_code': 20000, 'status_message': 'Ok.', 'time': '0.0749 sec.', 'cost': 0.05, 'tasks_count': 1, 'tasks_error': 0, 'tasks': [{'id': '01231404-7345-0110-0000-13f0ece301b4', 'status_code': 20100, 'status_message': 'Task Created.', 'time': '0.0018 sec.', 'cost': 0.05, 'result_count': 0, 'path': ['v3', 'keywords_data', 'google_ads', 'search_volume', 'task_post'], 'data': {'api': 'keywords_data', 'function': 'search_volume', 'se': 'google_ads', 'location_name': 'Brazil', 'keywords': ['chevrolet tracker premier 1.2', 'fiat fastback limited edition', 'honda hr-v exl', 'hyundai creta n-line', 'jeep renegade longitude t270', 'nissan kicks ex-clusive pack', 'renault duster iconic 1.3 tc']}, 'result': None}]}


## Get Request

In [6]:
response = client.get("/v3/keywords_data/google_ads/search_volume/tasks_ready")

if response['status_code'] == 20000:
    results = []
    for task in response['tasks']:
        if (task['result'] and (len(task['result']) > 0)):
            for resultTaskInfo in task['result']:
                # 2 - using this method you can get results of each completed task
                # GET /v3/keywords_data/google_ads/search_volume/task_get/$id
                if (resultTaskInfo['endpoint']):
                    results.append(client.get(resultTaskInfo['endpoint']))
                '''
                # 3 - another way to get the task results by id
                # GET /v3/keywords_data/google_ads/search_volume/task_get/$id              
                if(resultTaskInfo['id']):
                    results.append(client.get("/v3/keywords_data/google_ads/search_volume/task_get/" + resultTaskInfo['id']))
                '''
    print(results)
    # do something with result
else:
    print("error. Code: %d Message: %s" %
          (response["status_code"], response["status_message"]))

[{'version': '0.1.20231117', 'status_code': 20000, 'status_message': 'Ok.', 'time': '0 sec.', 'cost': 0, 'tasks_count': 1, 'tasks_error': 0, 'tasks': [{'id': '01231404-7345-0110-0000-13f0ece301b4', 'status_code': 20000, 'status_message': 'Ok.', 'time': '0 sec.', 'cost': 0, 'result_count': 7, 'path': ['v3', 'keywords_data', 'google_ads', 'search_volume', 'task_get', '01231404-7345-0110-0000-13f0ece301b4'], 'data': {'api': 'keywords_data', 'function': 'search_volume', 'se': 'google_ads', 'location_name': 'Brazil', 'keywords': ['chevrolet tracker premier 1.2', 'fiat fastback limited edition', 'honda hr-v exl', 'hyundai creta n-line', 'jeep renegade longitude t270', 'nissan kicks ex-clusive pack', 'renault duster iconic 1.3 tc']}, 'result': [{'keyword': 'chevrolet tracker premier 1.2', 'spell': None, 'location_code': 2076, 'language_code': None, 'search_partners': False, 'competition': 'LOW', 'competition_index': 22, 'search_volume': 50, 'low_top_of_page_bid': 0.24, 'high_top_of_page_bid':

## EDA

In [7]:
df = pd.DataFrame(results[0]['tasks'][0]['result'])
df.head()

,keyword,spell,location_code,language_code,search_partners,competition,competition_index,search_volume,low_top_of_page_bid,high_top_of_page_bid,cpc,monthly_searches
0,chevrolet tracker premier 1.2,None,2076,None,False,LOW,22.0,50.0,0.24,1.31,0.94,"[{'year': 2023, 'month': 12, 'search_volume': ..."
1,fiat fastback limited edition,None,2076,None,False,LOW,6.0,9900.0,0.04,0.28,0.29,"[{'year': 2023, 'month': 12, 'search_volume': ..."
2,honda hr-v exl,None,2076,None,False,LOW,11.0,4400.0,0.09,0.32,0.26,"[{'year': 2023, 'month': 12, 'search_volume': ..."
3,hyundai creta n-line,None,2076,None,False,LOW,19.0,1600.0,0.10,0.96,0.38,"[{'year': 2023, 'month': 12, 'search_volume': ..."
4,jeep renegade longitude t270,None,2076,None,False,LOW,30.0,320.0,0.31,1.91,1.20,"[{'year': 2023, 'month': 12, 'search_volume': ..."


In [8]:
monthly = dict(zip(df["keyword"], df["monthly_searches"]))
dfs_list = []
for key, value in monthly.items():
    temp_df = pd.DataFrame(value)
    temp_df["good"] = key
    dfs_list.append(temp_df)

df_monthly = pd.concat(dfs_list)
df_monthly["date"] = pd.to_datetime(
    df_monthly[['year', 'month']].assign(DAY=1))
df_monthly.head()

,year,month,search_volume,good,date
0,2023.0,12.0,50.0,chevrolet tracker premier 1.2,2023-12-01
1,2023.0,11.0,50.0,chevrolet tracker premier 1.2,2023-11-01
2,2023.0,10.0,40.0,chevrolet tracker premier 1.2,2023-10-01
3,2023.0,9.0,40.0,chevrolet tracker premier 1.2,2023-09-01
4,2023.0,8.0,40.0,chevrolet tracker premier 1.2,2023-08-01


In [9]:
# Viz - Vol
fig = px.line(data_frame=df_monthly, x="date", y="search_volume",
              color="good", title="Volume de Buscas por Modelo", markers=True)
fig.update_xaxes(tickformat="%b-%Y")
fig.show()

In [10]:
df_monthly.groupby(["year", "month"])["search_volume"].sum().values

array([22960., 19220., 23550., 19830., 17920., 13940., 16550., 18350.,
       13550., 15010.,  9260., 12920.])

In [11]:
df_monthly["relative_search"] = df_monthly["search_volume"] / \
    df_monthly.groupby(["year", "month"])["search_volume"].transform('sum')


df_monthly.head()

,year,month,search_volume,good,date,relative_search
0,2023.0,12.0,50.0,chevrolet tracker premier 1.2,2023-12-01,0.003870
1,2023.0,11.0,50.0,chevrolet tracker premier 1.2,2023-11-01,0.005400
2,2023.0,10.0,40.0,chevrolet tracker premier 1.2,2023-10-01,0.002665
3,2023.0,9.0,40.0,chevrolet tracker premier 1.2,2023-09-01,0.002952
4,2023.0,8.0,40.0,chevrolet tracker premier 1.2,2023-08-01,0.002180


In [13]:

# Viz - Share
fig = px.bar(data_frame=df_monthly, x="date", y="relative_search",
             color="good", title="Share de Buscas por Modelo", text_auto=".2%")
fig.update_xaxes(tickformat="%b-%Y")
fig.show()